# **Keyword Extraction**

## Instalasi dengan pip

Bagian ini menggunakan perintah `pip` untuk menginstal berbagai paket Python yang diperlukan

In [ ]:
!pip install selenium
!pip install networkx
!pip install matplotlib
!pip install nltk
!pip install Sastrawi
!pip install tqdm
!pip install pandas
!pip install sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## Import library dan download data NLTK

Bagian ini mengimpor berbagai library dan modul yang telah diinstal sebelumnya untuk digunakan dalam proyek

In [ ]:
import pandas as pd
import re

from tqdm import tqdm

import pickle
import nltk
import networkx as nx
import matplotlib.pyplot as plt
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from collections import Counter
from itertools import combinations
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Scraping data dan masukan ke DataFrame

Bagian ini melakukan proses scraping data dari artikel web dan menyimpannya dalam sebuah DataFrame

In [ ]:
def web_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

def get_element_text(driver, xpath):
    try:
        return WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        ).text.strip()
    except Exception as e:
        print(f"Error finding element with XPath {xpath}: {e}")
        return ""

def extract_article_content(driver, article_url):
    driver.get(article_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//h1'))
    )
    title = get_element_text(driver, './/h1[@class="jeg_post_title"]')

    # XPath untuk tanggal
    date = get_element_text(driver, './/div[@class="jeg_meta_date"]')

    # XPath untuk konten
    content_elements = driver.find_elements(By.XPATH, './/div[@class="content-inner "]/p')
    content = " ".join(p.text for p in content_elements)

    # XPath untuk kategori
    kategori = get_element_text(driver, './/span[@class="breadcrumb_last_link"]/a')

    return {
        "Title": title,
        "Date": date,
        "Content": content,
        "Category": kategori  # Menyimpan kategori dalam hasil
    }

driver = web_driver()

# Meminta input dari pengguna untuk URL artikel
url_input = input("Masukkan URL artikel: ")

# Scraping data dari URL artikel yang diberikan oleh pengguna
article_data = extract_article_content(driver, url_input)

driver.quit()

# Simpan hasil scraping ke dalam DataFrame
data = pd.DataFrame([article_data])  # Mengonversi hasil ke dalam list untuk membuat DataFrame

Masukkan URL artikel: https://mojok.co/liputan/kuliner/warung-pecel-cepu-blora-dirindukan-di-amerika-dan-rusia/


## Proses pembersihan teks dan tokenisasi

Bagian ini melakukan serangkaian langkah pembersihan dan persiapan teks untuk analisis lebih lanjut

In [ ]:
# Fungsi untuk mengubah teks menjadi huruf kecil
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

data['lower case'] = data['Content'].apply(clean_lower)

# Fungsi untuk membersihkan tanda baca dan angka (menghapus koma)
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z .]+')  # Koma dihilangkan dari pengecualian
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

data['tanda baca'] = data['lower case'].apply(clean_punct)

# Fungsi untuk normalisasi spasi
def _normalize_whitespace(text):
    if isinstance(text, str):
        corrected = re.sub(r'\s+', ' ', text)
        return corrected.strip()
    return text

data['spasi'] = data['tanda baca'].apply(_normalize_whitespace)

# Tokenisasi
def tokenize_text(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return text

data['token'] = data['spasi'].apply(tokenize_text)

# Pembersihan stopwords setelah tokenisasi
def clean_stopwords(tokens):
    if isinstance(tokens, list):
        stopword = set(stopwords.words('indonesian'))
        filtered_tokens = [word for word in tokens if word not in stopword]
        return filtered_tokens
    return tokens

data['stopwords'] = data['token'].apply(clean_stopwords)

# Gabungkan kembali token menjadi string
data['processed_text'] = data['stopwords'].apply(lambda tokens: ' '.join(tokens) if isinstance(tokens, list) else '')

## Proses tokenisasi lanjutan

Bagian ini melanjutkan proses pembersihan dan tokenisasi teks dengan fokus pada pemecahan teks menjadi kalimat dan kata-kata per kalimat

In [ ]:
def clean_lower(text):
    if isinstance(text, str):
        return text.lower()
    return text

data['lower_case'] = data['Content'].apply(clean_lower)

# Fungsi untuk membersihkan tanda baca dan angka (menghapus koma)
def clean_punct(text):
    if isinstance(text, str):
        clean_patterns = re.compile(r'[0-9]|[/(){}\[\]\|@,;_]|[^a-z .]+')  # Koma dihilangkan dari pengecualian
        text = clean_patterns.sub(' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return text

data['clean_punct'] = data['lower_case'].apply(clean_punct)

# Tokenisasi kalimat
def split_sentences(text):
    if isinstance(text, str):
        return sent_tokenize(text)
    return []

data['kalimat'] = data['clean_punct'].apply(split_sentences)

# Tokenisasi kata di setiap kalimat
def split_words(sentences):
    if isinstance(sentences, list):
        return [word_tokenize(sentence) for sentence in sentences]
    return []

data['kata_per_kalimat'] = data['kalimat'].apply(split_words)

# Pembersihan stopwords untuk setiap kata
def clean_stopwords_per_sentence(token_list):
    if isinstance(token_list, list):
        stopword = set(stopwords.words('indonesian'))
        return [[word for word in sentence if word not in stopword] for sentence in token_list]
    return token_list

data['stopwords'] = data['kata_per_kalimat'].apply(clean_stopwords_per_sentence)

data[['kalimat', 'kata_per_kalimat', 'stopwords']]

,kalimat,kata_per_kalimat,stopwords
0,[tugas lapangan di cepu blora membawa saya pad...,"[[tugas, lapangan, di, cepu, blora, membawa, s...","[[tugas, lapangan, cepu, blora, membawa, warun..."


## Pengumpulan kalimat dan vocabulary

Bagian ini mengumpulkan semua kalimat dari teks yang telah diproses dan membangun kumpulan kata unik (vocabulary) untuk analisis lebih lanjut

In [ ]:
# Membuat kumpulan kalimat
sentences = []
for p in data['processed_text']:
  sentences.extend(sent_tokenize(p))

# Membuat kumpulan kata unik
vocabulary = set()
for sentence in sentences:
  for word in sentence.split():
    vocabulary.add(word)

print(sentences)
print(vocabulary)

['tugas lapangan cepu blora membawa warung pecel legendaris .', 'namanya warung pecel sunti lokasinya jl .', 'st. kota .', 'warung kerap mengirim bumbu amerika rusia .', '.', 'wib suasana warung pecel sunti berjejal pembeli kamis .', 'masuk .', 'antre pembeli pembeli memilih membungkus .', 'orang laki laki berseragam kerja pekerja pertambangan minyak cepu blora makan .', 'telas napa mboten bu habis nggak bu nasi boran tinggal mencoba .', 'tenang mas paruh baya bernama tatik meladeni pembeli pembeli antre .', 'tatik grapyak pembeli .', 'jarang melempar gojlokan berujung gojlok gelak tawa .', 'teman teman konten konten lek damis ya sosok tatik .', 'orang cepu kaget mengiranya jutek .', 'giliran .', 'pengelola warung pecel sunti cepu blora menyodorkan sepiring nasi pecel lauk telur tempe .', 'diguyur lodeh nggak mas .', 'tawari barang kali masnya nggak cocok tawar tatik .', 'mengangguk salahnya .', 'dasarnya makan .', 'halal kenyang .', 'lantas meraih piring mengguyurnya sayur lodeh .', '

## Fungsi vektorisasi kata

Fungsi `vektor_kata` bertujuan untuk mengonversi kumpulan kalimat menjadi representasi vektorial berdasarkan frekuensi kata

In [ ]:
def vektor_kata(data):
	vektor_kata = pd.DataFrame(0, index=range(len(data)), columns=vocabulary)

	for i, sent in enumerate(data):
		# Tokenisasi kalimat menjadi kata-kata
		kata_kalimat = word_tokenize(sent)

		# Hitung frekuensi setiap kata dalam kalimat
		for word in kata_kalimat:
			if word in vocabulary:
				vektor_kata.at[i, word] += 1

	return vektor_kata

## Pembuatan dataframe dengan word counts

Bagian ini membangun sebuah DataFrame yang menyimpan jumlah kemunculan setiap kata dalam setiap kalimat

In [ ]:
pd.set_option('future.no_silent_downcasting', True)
df = pd.DataFrame(columns=list(vocabulary), index=sentences)

# Mengisi nilai kosong dengan 0
df = df.fillna(0)  # Fill with 0s

# Menghitung nilai setiap kata pada kalimat
for i, sentence in enumerate(sentences):
  for word in sentence.split():
    df.loc[sentence, word] += 1

# Melihat isi Dataframe
df

,muchamad,halal,damis,pertambangan,daerah,gurih,napa,gelak,sisanya,sosok,...,pas,mohon,giliran,blora,cepu,timpal,meladeni,telur,kendali,sahur
tugas lapangan cepu blora membawa warung pecel legendaris .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
namanya warung pecel sunti lokasinya jl .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
st. kota .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
warung kerap mengirim bumbu amerika rusia .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
gurih timpal tatik .,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
berpamitan tatik mampir warung pecel legendaris cepu blora .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
warung pecel sunti buka .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
libur tatik capek .,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Perhitungan frekuensi kata dan ekstraksi kata kunci

Bagian ini menghitung frekuensi keseluruhan setiap kata dalam korpus teks dan mengekstrak kata-kata kunci teratas berdasarkan frekuensi tersebut

In [ ]:
# Menghitung jumlah kemunculan pada tiap kata
word_frequencies = df.sum(axis=0)

# Mengurutkan berdasarkan jumlah kemunculan kata
sorted_word_frequencies = word_frequencies.sort_values(ascending=False)

# Menampilkan kata teratas
keyword = 3

for i, (word, freq) in enumerate(sorted_word_frequencies.items()):
    if i < keyword:
        print(f"Rank {i+1}: {word} (Frequency: {freq})")

Rank 1: . (Frequency: 100)
Rank 2: tatik (Frequency: 31)
Rank 3: pecel (Frequency: 21)
